In [142]:
import numpy as np
np.set_printoptions(precision=4, suppress=True, linewidth=200)
np.random.seed(2018)
import pandas as pd
import tensorflow as tf
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 6.0) #set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
sns.set(color_codes=True)
np.set_printoptions(precision=10, suppress=True, linewidth=200, threshold=1000, edgeitems=25)
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1", "2' "3" "4"
os.environ["CUDA_VISIBLE_DEVICES"]= ''
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
pd.set_option('display.float_format', lambda x: '{:.2f}'.format(x))

import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [143]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.preprocessing import Normalizer, MinMaxScaler

from scipy.cluster.vq import kmeans2 as kmeans

In [160]:
import pickle

In [144]:
import data_loader

In [148]:
dtrain, dtest, z, y_std = data_loader.load('airplane.csv', 
                                           n_clusters=10, 
                                           n_induce=10)

loading data of 2055732 datapoints...
Parition into 10 clusters...
Done...!
Selecting 10 inducing variables...
Done...!


In [149]:
z

array([[-0.0179284717,  0.0019335782,  0.017862637 ,  0.2110224373,  0.1670444271, -0.0526684079,  2.4312289238,  2.4658456743],
       [-0.9999493627,  0.5615597869, -0.3055145105,  0.8547245697,  0.8587486505, -0.2506033733, -0.3674875351, -0.3720032212],
       [ 0.0705374218, -0.9954600425,  0.5040008111,  0.8869293463,  0.8928602894, -0.2370555461, -0.353768255 , -0.3522754015],
       [ 0.9408237903,  0.6457341188, -0.3190887082,  0.9067389244,  0.9326147271, -0.2204848435, -0.2546047288, -0.2525597503],
       [-0.7359712287,  0.8199812097, -0.0630753305, -0.6869368361, -0.5658075472, -0.1203727054,  0.6138396752,  0.5950210642],
       [-1.0060037805, -0.5949306288, -0.1531476526, -0.7756301883, -0.8292271875, -0.1535441572, -0.5056619261, -0.5087889811],
       [ 0.6563558827, -0.8874062092,  0.287212931 , -0.6489287883, -0.5280525907, -0.0697709619,  0.7218450696,  0.7107884736],
       [-0.1141023363, -0.0486756423, -0.0506764632,  0.2342179982,  0.2673529397,  2.1323411548,

In [150]:
X_train, y_train = dtrain[:, :-2], dtrain[:, -2:-1]
X_test, y_test = dtest[:, :-2], dtest[:, -2:-1]

## VBSGPR

In [294]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [295]:
with open('svgp.pkl', 'rb') as f:
    dic = pickle.load(f)

In [296]:
import tensorflow as tf
import numpy as np
import random
from model import VBSGPR

epochs = 10
N, total_dim = dtest.shape
log_beta, log_sf2, log_theta = dic['log_beta_opt'], dic['log_sf2_opt'], dic['log_theta_opt']
clusters = [i for i in range(100)]

In [297]:
model = VBSGPR(N, log_beta, log_sf2, log_theta, 
               z=dic['z_opt'], qmu=dic['qmu_opt'],
               qlogdev=dic['qlogdev_opt'], whiten=True)

In [298]:
lb = model.lower_bound()
fmu, fcov = model.predict_f()
gp_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'vbsgpr')
gp_opt = tf.train.AdamOptimizer(0.01, name='gp_opt')
gp_train_op = gp_opt.minimize(-lb, var_list=gp_vars)

In [299]:
sess.run(tf.global_variables_initializer())
# f_test, _ = sess.run([fmu, fcov], {model.x: X_test[0:1, :]})
# rmse = np.sqrt(np.mean(y_std**2 * ((y_test - f_test))**2))
# print ('test RMSE: {}'.format(rmse))

In [300]:
sess.run(lb, {model.x: X_test, model.y: y_test, model.batch: y_test.shape[0]})

-28889.615

In [292]:
f_test, f_cov = sess.run([fmu, fcov], {model.x: X_test})

In [293]:
np.sqrt(np.mean(y_std**2 * ((y_test - f_test))**2))

40.05194397437414